# Обучение модели и подсчет метрик

### Выбор модели и аргументы в пользу неё

Поскольку наша задача выдления основных трендов и тем, я полагаю, что использование ``Bertopic`` является подходящим решением, поскольку

- Он использует современные языковые модели для получения контекстуальных эмбеддингов, что важно для специализированных текстов.

- Кластеризация с ``HDBSCAN`` эффективно отделяет релевантные темы от шума.

- Автоматическое извлечение ключевых слов с помощью ``TF-IDF`` позволяет формировать интерпретируемые названия тем.

- Возможность интегрировать временной анализ помогает визуализировать динамику трендов по месяцам.


### Каким образом работает модель

Общая интиуция тематического моделирования с помощью ``Bertopic``:

1. **Предобработка данных**: выбор текстов для тематического моделирования, препроцесим тексты (удалеям стоп слова, чистим http сылки и тп), но важно, что преобразование в эмбединги идет без удаления стоп слов 

2. **Получение эмбеддингов**: с помощью энкодера получаем векторное представление текстов

3. **Снижение размерности**: С помощью umap, мы снижаем размерность для кластеризации, чтобы минимизировать проклятие размерности, используем umap, а не pca, поскольку эмбединги нелинейные

3. **Кластеризация**: Применяем алгоритм кластеризации, в нашем случае ``HDBSCAN``, поскольку как я говорил ранее нам хотелось бы отделить шумовые кластеры

4. **Получение названий топиков**: Для каждого кластера автоматически извлекаем ключевые слова с помощью TF-IDF

5. **Оцениваем качество модели**: Используем метрки качетсва кластеризации, а также топиков.


## Технические особенности

Из коробки ``Bertopic`` работает на CPU, поэтому я использовал приприетарную библиотеку Nvidia ``cuml`` для переноса вычисления на гпу, также препроцесинг текстов реализован многопоточно с помощью библиотеки ``pandarallel``.

In [2]:
import pandas as pd
import re
import torch
from numba import cuda
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import bertopic
from sklearn.feature_extraction.text import CountVectorizer
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
import os
from pandarallel import pandarallel
import spacy
from utils import check_gpu
import random

random.seed(42)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tqdm.pandas()
check_gpu()

Доступно GPU: 1
Название GPU: NVIDIA GeForce RTX 4070 Ti SUPER


загрузим данные из ноутбука ``2_data_observation.ipynb``

In [3]:
df = pd.read_csv('data/video_generation_2024.csv')
df.sample(1)

,entry_id,arxiv_id,title,authors,abstract,published,updated,year,categories,primary_category,pdf_url,arxiv_url,doi,comment,journal_ref
2518,http://arxiv.org/abs/2409.07236v2,2409.07236v2,3DGCQA: A Quality Assessment Database for 3D A...,"Yingjie Zhou, Zicheng Zhang, Farong Wen, Jun J...",Although 3D generated content (3DGC) offers ad...,2024-09-11,2024-09-12,2024,"eess.IV, cs.CV",eess.IV,http://arxiv.org/pdf/2409.07236v2,http://arxiv.org/abs/2409.07236v2,NaN,NaN,NaN


### Предобработка данных

Я заметил, что в abstract большое количество ссылок, которые будут зашумлять данные, давайте их почистим, хотя для современных энкодеров препроцессинг особо не нужен.

In [4]:
# Remove links from the abstracts
df['abstract'] = df['abstract'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

Также я полагаю эффективным использование такой фичи как название статьи + abstract, пскольку это даст больше информации модели.

In [5]:
# Make new column with title and abstract to use for clustering
df['title_abstract'] = df['title'] + ' ' + df['abstract']

### Получение эмбеддингов

Энкодер я выбрал ``all-MiniLM-L6-v2``, поскольку он относительно легковесный для моей Видеокарты и является отчасти отрослевым бейзлайном

In [6]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(df.title_abstract.to_list(), show_progress_bar=True, batch_size=64)

Batches:   0%|          | 0/109 [00:00<?, ?it/s]

### Препроцессинг для получение названий топиков

Также я решил лематизировать тексты, поскольку из коробки ``tf-idf`` бертопика работает не очень хорошо, давайте упростим себе и ему задачу

In [7]:
def lemmatize(text):
    if not hasattr(lemmatize, "nlp"):
        lemmatize.nlp = spacy.load("en_core_web_lg", disable=["parser", "ner"])
        
    doc = lemmatize.nlp(text)
    return " ".join(token.lemma_ for token in doc if not token.is_punct and not token.is_space)

pandarallel.initialize(nb_workers=8, progress_bar=True)

df['title_abstract_lem'] = df['title_abstract'].parallel_apply(lemmatize)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Помимо этого я решил убрать стоп слова и добавить в них стоп слова, которые характерны для научных статей.

In [8]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
standard_stop_words = set(stopwords.words('english'))

scientific_stop_words = set([
    'introduction', 'conclusion', 'method', 'methods', 'result', 'results',
    'discussion', 'paper', 'article', 'journal', 'figure', 'fig', 'table',
    'author', 'study', 'studies', 'analysis', 'data', 'based', 'using', 'used',
    'show', 'shown', 'propose', 'proposed', 'approach', 'demonstrate', 'demonstrated',
    'experiment', 'experimental', 'conclude', 'research', 'review', 'literature',
    'abstract', 'keywords', 'acknowledgement', 'model', 'models', 'methodology',
    'performance', 'related', 'work', 'previous', 'recent', 'technique',
    'system', 'systems', 'evaluate', 'evaluated', 'evaluation', 'state', 'art',
    'significant', 'contribution', 'contributions', 'novel', 'effective', 'effectively'
])

final_stop_words = standard_stop_words.union(scientific_stop_words)

vectorizer_model = CountVectorizer(
    stop_words=list(final_stop_words), 
    max_df=0.8,
    min_df=2
)

[nltk_data] Downloading package stopwords to /home/kiril/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Снижение размерности

In [9]:
umap_model = UMAP(
    n_components=15,
    n_neighbors=15,
    min_dist=0.05,
    metric='cosine',
    random_state=42
)

[2025-04-02 02:46:32.495] [CUML] [info] build_algo set to brute_force_knn because random_state is given


### Кластеризация

In [10]:
hdbscan_model = HDBSCAN(
    min_cluster_size=25,
    min_samples=8,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True,
)

### Формирование модели bertopic

In [11]:
bertopic_model = bertopic.BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    verbose=True,
    calculate_probabilities=True,
    top_n_words=3,
)

### Обучение модели

In [12]:
topics, probs = bertopic_model.fit_transform(df.title_abstract_lem.to_list(), embeddings=embeddings)

topic_info = bertopic_model.get_topic_info()

print(f"Обнаружено кластеров: {len(topic_info)}")
print(f"Информация о кластерах:")
topic_info[['Topic', 'Count', 'Name']]

2025-04-02 02:46:32,571 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-02 02:46:33,253 - BERTopic - Dimensionality - Completed ✓
2025-04-02 02:46:33,254 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-02 02:46:33,560 - BERTopic - Cluster - Completed ✓
2025-04-02 02:46:33,563 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-04-02 02:46:33,921 - BERTopic - Representation - Completed ✓


Обнаружено кластеров: 35
Информация о кластерах:


,Topic,Count,Name
0,-1,2132,-1_scene_camera_action
1,0,1364,0_segmentation_medical_mri
2,1,517,1_editing_edit_t2v
3,2,414,2_robot_policy_manipulation
4,3,202,3_facial_head_talk
5,4,197,4_gaussian_scene_splatting
6,5,169,5_deepfake_attack_adversarial
7,6,152,6_super_resolution_sr
8,7,129,7_traffic_forecasting_graph
9,8,110,8_quantum_vortex_wave


### Подсчет качества кластеризации и тематического моделирования

Краткая справка по метрикам: 

``Silhouette Score``: Оценивает разделение кластеров  (от -1 до 1, хорошие > 0.5)

``c_v``: Мера семантической связности темы  (от 0 до 1, хорошие > 0.5)

``c_npmi``: Нормированная точечная взаимная информация между словами  (от -1 до 1, хорошие > 0)

``u_mass``: Когерентность на основе со-встречаемости  (от -∞ до 0, хорошие – ближе к 0)

``c_uci``: Мера когерентности через точечную взаимную информацию (чем выше – тем лучше).

``Topic Diversity``: Доля уникальных слов в топ-N тем (от 0 до 1, хорошие – ближе к 1)

In [ ]:
import numpy as np
import cupy as cp
from cuml.metrics.cluster import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
import ast
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

topic_info = bertopic_model.get_topic_info() 
topic_info = topic_info[topic_info.Topic != -1]
topic_words = topic_info['Representation'].tolist()
topic_words = [ast.literal_eval(rep) if isinstance(rep, str) else rep for rep in topic_words]

N = 3

umap_embeddings = umap_model.transform(embeddings)

labels = np.array(topics)
mask = labels != -1

umap_embeddings_gpu = cp.asarray(umap_embeddings[mask])
labels_gpu = cp.asarray(labels[mask])

sil_score = silhouette_score(umap_embeddings_gpu, labels_gpu, metric='cosine')
print("Silhouette Score:", sil_score)

docs = [doc.split() for doc in df['title_abstract_lem']]

dictionary = Dictionary(docs)

coherence_cv    = CoherenceModel(topics=topic_words, texts=docs, dictionary=dictionary, coherence='c_v').get_coherence()
coherence_npmi  = CoherenceModel(topics=topic_words, texts=docs, dictionary=dictionary, coherence='c_npmi').get_coherence()
coherence_umass = CoherenceModel(topics=topic_words, texts=docs, dictionary=dictionary, coherence='u_mass').get_coherence()
coherence_uci   = CoherenceModel(topics=topic_words, texts=docs, dictionary=dictionary, coherence='c_uci').get_coherence()

print("Coherence (c_v):", coherence_cv)
print("Coherence (c_npmi):", coherence_npmi)
print("Coherence (u_mass):", coherence_umass)
print("Coherence (c_uci):", coherence_uci)


unique_topic_words = set(word for topic in topic_words for word in topic[:N])
topic_diversity = len(unique_topic_words) / (len(topic_words) * N)
print("Topic Diversity:", topic_diversity)

Silhouette Score: 0.5397239923477173
Coherence (c_v): 0.8210221614838007
Coherence (c_npmi): 0.19905209113081204
Coherence (u_mass): -2.8766630095168915
Coherence (c_uci): 0.8559725662682557
Topic Diversity: 0.9411764705882353


### Cохранение модели

In [ ]:
os.mkdir('bertopic_model', exist_ok=True)

bertopic_model.save(path='bertopic_model', serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

### Cохранеяем препроценный датасет

In [15]:
df.to_csv('data/video_generation_2024_bertopic.csv', index=False)